In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_train.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_train_comfort_128.joblib")

In [2]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/comfort_train_processed.csv")

In [3]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["pos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 1 1 ... 1 0 0]
[0 1 1 ... 1 0 0]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[0]


In [4]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention

In [6]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
#att = SeqWeightedAttention()(bi)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       use_attention_bias=True,return_attention=False) (bi)
flat = Flatten()(att)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)
m.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])





In [7]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 768)]   0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 128, 64)      213248      ['input_1[0][0]']                
                                                                                                  
 seq_self_attention (SeqSelfAtt  (None, 128, 64)     4097        ['lstm[0][0]']                   
 ention)                                                                                          
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                           

In [8]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=5)

Epoch 1/5
21/21 [==============================] - 24s 985ms/step - loss: 0.3610 - accuracy: 0.8697
Epoch 2/5
21/21 [==============================] - 20s 972ms/step - loss: 0.1803 - accuracy: 0.9449
Epoch 3/5
21/21 [==============================] - 13s 583ms/step - loss: 0.1205 - accuracy: 0.9637
Epoch 4/5
21/21 [==============================] - 12s 567ms/step - loss: 0.0916 - accuracy: 0.9760
Epoch 5/5
21/21 [==============================] - 12s 567ms/step - loss: 0.0851 - accuracy: 0.9783


In [9]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/comfort_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_comfort_128.joblib")

In [10]:
# define example
data = dataset_test["pos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
##print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
#print(onehot_encoded_test)

[0 0 0 ... 1 1 0]


In [11]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [12]:
print(predictions)

[[9.9793649e-01 2.0635603e-03]
 [9.9793649e-01 2.0635603e-03]
 [9.8433244e-01 1.5667567e-02]
 ...
 [2.9571562e-03 9.9704283e-01]
 [8.9889736e-06 9.9999106e-01]
 [9.4732541e-01 5.2674618e-02]]


In [13]:
import numpy as np
predictions_vec = np.zeros((1171,4))

In [14]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [15]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [16]:
len(labels_to_eval)

1171

In [17]:
dataset_test["predicted_label_neg"] = labels_to_eval
dataset_test["confidence_label_neg"] = confidences

In [18]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["pos"],labels_to_eval,digits=5))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0    0.92260   0.90030   0.91131       662
           1    0.87429   0.90177   0.88781       509

    accuracy                        0.90094      1171
   macro avg    0.89844   0.90104   0.89956      1171
weighted avg    0.90160   0.90094   0.90110      1171



In [ ]:
dataset_test

In [ ]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_2_neg_absita.h5")

In [ ]:
#dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_neg_full_model2.csv");